In [1]:
import sys 
sys.path.append('../')
from itertools import repeat
import pandas as pd 
import numpy as np 
from scipy.stats import mode
import talib
import tulipy
import sqlite3 as sql
from p_tqdm import p_map, p_umap, t_map
from tqdm.notebook import tqdm_notebook
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, v in enumerate(my_assets):
    print(i, '|', v)

0 | EUR/USD
1 | USD/JPY
2 | GBP/USD
3 | USD/CHF
4 | AUD/USD
5 | USD/CAD
6 | NZD/USD
7 | Bund
8 | AUS200
9 | ESP35
10 | EUSTX50
11 | FRA40
12 | GER30
13 | HKG33
14 | JPN225
15 | NAS100
16 | SPX500
17 | UK100
18 | US30
19 | Copper
20 | NGAS
21 | UKOil
22 | USOil
23 | XAU/USD
24 | XAG/USD


In [3]:
freq = 'H1'
sql_path = f'PriceData_{freq}.db'
strategies = ['SMA', 'EMA', 'KAMA', 'DEMA', 'TEMA', 
              'TRIMA', 'HMA', 'WMA', 'ZLEMA', 'TSF',
              'WILDERS', 'SLOPE', 'TRIX']
parameters = []
for i in np.arange(10, 101, 10):
    for j in np.arange(10, 101, 10):
        if i < j: parameters.append((i,j))
print(f'Parameters length: {len(parameters)}')

Parameters length: 45


In [4]:
def get_signals(data, param, start, end, strategy):
    if param[0] in [None, np.nan]: return np.nan
    if param[1] in [None, np.nan]: return np.nan
    
    data = data.copy(deep=True)
    try:
        # 1
        if strategy == 'SMA':
            data["param1"] = talib.SMA(data.close, param[0])
            data["param2"] = talib.SMA(data.close, param[1])
        
        # 2
        elif strategy == 'EMA':
            data["param1"] = talib.EMA(data.close, param[0])
            data["param2"] = talib.EMA(data.close, param[1])
        
        # 3
        elif strategy == 'KAMA':
            data["param1"] = talib.KAMA(data.close, param[0])
            data["param2"] = talib.KAMA(data.close, param[1])
        
        # 4
        elif strategy == 'DEMA':
            data["param1"] = talib.DEMA(data.close, param[0])
            data["param2"] = talib.DEMA(data.close, param[1])
        
        # 5
        elif strategy == 'TEMA':
            data["param1"] = talib.TEMA(data.close, param[0])
            data["param2"] = talib.TEMA(data.close, param[1])
        
        # 6
        elif strategy == 'TRIMA':
            data["param1"] = talib.TRIMA(data.close, param[0])
            data["param2"] = talib.TRIMA(data.close, param[1])
        
        # 7
        elif strategy == 'HMA':
            data["param1"] = np.nan
            param1 = tulipy.hma(data.close.values, param[0])
            data.iloc[-len(param1):,data.columns.get_loc("param1")] = param1
            data["param2"] = np.nan
            param2 = tulipy.hma(data.close.values, param[1])
            data.iloc[-len(param2):,data.columns.get_loc("param2")] = param2
        
        # 8
        elif strategy == 'WMA':
            data["param1"] = talib.WMA(data.close, param[0])
            data["param2"] = talib.WMA(data.close, param[1])
        
        # 9
        elif strategy == 'ZLEMA':
            data["param1"] = np.nan
            param1 = tulipy.zlema(data.close.values, param[0])
            data.iloc[-len(param1):,data.columns.get_loc("param1")] = param1
            data["param2"] = np.nan
            param2 = tulipy.zlema(data.close.values, param[1])
            data.iloc[-len(param2):,data.columns.get_loc("param2")] = param2
        
        # 10
        elif strategy == 'TSF':
            data["param1"] = np.nan
            param1 = tulipy.tsf(data.close.values, param[0])
            data.iloc[-len(param1):,data.columns.get_loc("param1")] = param1
            data["param2"] = np.nan
            param2 = tulipy.tsf(data.close.values, param[1])
            data.iloc[-len(param2):,data.columns.get_loc("param2")] = param2
        
        # 11    
        elif strategy == 'WILDERS':
            data["param1"] = np.nan
            param1 = tulipy.wilders(data.close.values, param[0])
            data.iloc[-len(param1):,data.columns.get_loc("param1")] = param1
            data["param2"] = np.nan
            param2 = tulipy.wilders(data.close.values, param[1])
            data.iloc[-len(param2):,data.columns.get_loc("param2")] = param2
        
        # 12
        elif strategy == 'SLOPE':
            data["param1"] = np.nan
            param1 = tulipy.linregslope(data.close.values, param[0])
            data.iloc[-len(param1):,data.columns.get_loc("param1")] = param1
            data["param2"] = np.nan
            param2 = tulipy.linregslope(data.close.values, param[1])
            data.iloc[-len(param2):,data.columns.get_loc("param2")] = param2
        
        # 13
        elif strategy == 'TRIX':
            data["param1"] = np.nan
            param1 = tulipy.trix(data.close.values, param[0])
            data.iloc[-len(param1):,data.columns.get_loc("param1")] = param1
            data["param2"] = np.nan
            param2 = tulipy.trix(data.close.values, param[1])
            data.iloc[-len(param2):,data.columns.get_loc("param2")] = param2
            
    except:
        return np.nan
    data.dropna(axis=0, inplace=True)
    data['signal'] = np.where(data["param1"] > data["param2"], 1,
                               np.where(data["param1"] < data["param2"], -1, np.nan)
                              )
    data['signal'] = data['signal'].ffill()
    data = data.loc[start:end]
    if len(data) == 0:
        return np.nan
    else:
        return data


def get_strategy_return(data, param, freq, start, end, strategy):    
    data = get_signals(data, param, start, end, strategy)
    if not isinstance(data, pd.DataFrame): return (param, np.nan)
    if len(data) == 0: return (param, np.nan)
    backtester = IterativeBacktester(data=data, signals=data.signal, freq=freq)
    backtester.backtest(progress_bar=False)

    return param, backtester.return_df.loc['TotalReturn', 'Portfolio']


def get_optim_param(data, params, freq, start, end, strategy): 
    try:
        res = t_map(get_strategy_return, repeat(data), params, repeat(freq), 
                     repeat(start), repeat(end), repeat(strategy), leave=False)

        res = pd.DataFrame(res, columns=['params', 'returns'])
        res.sort_values(by='returns', ascending=False, inplace=True)
        best = res.iloc[0]
    except:
        return
    
    return start, end, best['params'], best['returns'] 

In [5]:
def get_optimized_args(ticker, strategy, parameters):

    # load data
    conn = sql.connect(sql_path)
    data_sql = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
    conn.close()
    for col in ['open', 'close', 'high', 'low']:
        data_sql[col] = data_sql[['bid'+col, 'ask'+col]].mean(axis=1)
        
    # get weekly dates
    sat_index = data_sql.resample('W-Sat').last().index
    start_dates = []
    end_dates = []
    for i in range(1, len(sat_index)-1):
        start_dates.append(sat_index[i])
        end_dates.append(sat_index[i+1])
        
    # get optimized params
    optim_params = p_umap(get_optim_param, 
                          repeat(data_sql), 
                          repeat(parameters), 
                          repeat(freq), 
                          start_dates, 
                          end_dates, 
                          repeat(strategy), 
                          leave=False,
                          desc=f'{strategy}-optimizing')
    opt_df = pd.DataFrame(optim_params, columns=['starts', 'ends', 'params', 'returns'])
    opt_df.sort_values(by='starts', ascending=True, inplace=True)
    opt_df.reset_index(inplace=True, drop=True)
    opt_df['param1'] = opt_df.params.apply(lambda x: x[0] if x is not None else None)
    opt_df['param2'] = opt_df.params.apply(lambda x: x[1] if x is not None else None)
    
    n_periods = [i for i in range(1, int(len(start_dates)*0.7), 1)]
    results = []
    for n in tqdm_notebook(n_periods, leave=False, desc=f'{strategy}-rolling periods'):
        param1_roll_median = opt_df.param1.iloc[:-1].rolling(n).median()
        param1_roll_mean = opt_df.param1.iloc[:-1].rolling(n).mean()
        param1_roll_mode = opt_df.param1.iloc[:-1].rolling(n).apply(lambda x: mode(x)[0])

        param2_roll_median = opt_df.param2.iloc[:-1].rolling(n).median()
        param2_roll_mean = opt_df.param2.iloc[:-1].rolling(n).mean()
        param2_roll_mode = opt_df.param2.iloc[:-1].rolling(n).apply(lambda x: mode(x)[0])

        # median
        param1_p = param1_roll_median.values
        param2_p = param2_roll_median.values
        param2_p = np.where(param2_p < 0.01, 0.01, param2_p)
        params = [*zip(param1_p, param2_p)]
        test_returns =  p_umap(get_strategy_return, 
                               repeat(data_sql), 
                               params, 
                               repeat(freq), 
                               start_dates[-len(params):], 
                               end_dates[-len(params):], 
                               repeat(strategy),
                               leave=False,
                               desc='median')
        test_returns = pd.DataFrame(test_returns, columns=['params', 'returns'])
        median_winrate = test_returns.returns.apply(np.sign).value_counts(normalize=True)[1]
        median_ret = test_returns.returns.mean()
        results.append(('median', n, median_winrate, median_ret))

        # mean
        param1_p = param1_roll_mean.values
        param2_p = param2_roll_mean.values
        param2_p = np.where(param2_p < 0.01, 0.01, param2_p)
        params = [*zip(param1_p, param2_p)]
        test_returns =  p_umap(get_strategy_return, 
                               repeat(data_sql), 
                               params, 
                               repeat(freq), 
                               start_dates[-len(params):], 
                               end_dates[-len(params):], 
                               repeat(strategy),
                               leave=False,
                               desc='mean')
        test_returns = pd.DataFrame(test_returns, columns=['params', 'returns'])
        mean_winrate = test_returns.returns.apply(np.sign).value_counts(normalize=True)[1]
        mean_ret = test_returns.returns.mean()
        results.append(('mean', n, mean_winrate, mean_ret))

        # mode
        param1_p = param1_roll_mode.values
        param2_p = param2_roll_mode.values
        param2_p = np.where(param2_p < 0.01, 0.01, param2_p)
        params = [*zip(param1_p, param2_p)]
        test_returns =  p_umap(get_strategy_return, 
                               repeat(data_sql), 
                               params, 
                               repeat(freq), 
                               start_dates[-len(params):], 
                               end_dates[-len(params):], 
                               repeat(strategy),
                               leave=False,
                               desc='mode')
        test_returns = pd.DataFrame(test_returns, columns=['params', 'returns'])
        mode_winrate = test_returns.returns.apply(np.sign).value_counts(normalize=True)[1]
        mode_ret = test_returns.returns.mean()
        results.append(('mode', n, mode_winrate, mode_ret))
        
    res_df = pd.DataFrame(results, columns = ['agg_methods', 'n_periods', 'win_rates', 'returns'])
    res_df.sort_values(by='returns', ascending=False, inplace=True)
    res_df.reset_index(inplace=True, drop=True)
    
    pos_ret = res_df.query('returns > 0')
    if len(pos_ret) > 0:
        best_agg = pos_ret.loc[0].agg_methods
        best_n = pos_ret.loc[0].n_periods
        winrate = pos_ret.loc[0].win_rates
        ret = pos_ret.loc[0].returns
        return (strategy, best_agg, best_n, winrate, ret)
    else:
        return (strategy, None, np.nan, np.nan, np.nan)

In [6]:
ticker_args = {}
for ticker in tqdm_notebook(my_assets, desc='tickers'):
    res = t_map(get_optimized_args, repeat(ticker), strategies, repeat(parameters), desc=ticker, leave=False)
    res_df = pd.DataFrame(res, columns=['strategy', 'agg', 'n_periods', 'winrates', 'rets'])
    res_df.sort_values(by='rets', ascending=False, inplace=True)
    res_df.reset_index(drop=True, inplace=True)
    ticker_args[ticker] = res_df.copy(deep=True)

tickers:   0%|          | 0/25 [00:00<?, ?it/s]

EUR/USD:   0%|          | 0/13 [00:00<?, ?it/s]

SMA-optimizing:   0%|          | 0/80 [00:00<?, ?it/s]

SMA-rolling periods:   0%|          | 0/55 [00:00<?, ?it/s]

median:   0%|          | 0/79 [00:00<?, ?it/s]

...

mode:   0%|          | 0/79 [00:00<?, ?it/s]

median:   0%|          | 0/79 [00:00<?, ?it/s]

Process ForkPoolWorker-831:
Process ForkPoolWorker-826:
Process ForkPoolWorker-832:
Process ForkPoolWorker-829:
Process ForkPoolWorker-825:
Process ForkPoolWorker-828:
Process ForkPoolWorker-830:
Traceback (most recent call last):
Process ForkPoolWorker-827:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _

  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/pandas/core/indexing.py", line 1520, in _getitem_axis
    self._validate_integer(key, axis)
KeyboardInterrupt
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/pandas/core/generic.py", line 1145, in _rename
    indexer = ax.get_indexer_for(replacements)

KeyboardInterrupt

  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/pandas/core/indexing.py", line 1450, in _validate_integer
    len_axis = len(self.obj._get_axis(axis))
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/pandas/core/indexing.py", line 1731, in _setitem_with_indexer_split_path
    self._setitem_single_column(ilocs[0], value, pi)
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 5764, in get_indexer_for
    return self.get_indexer(target)
  File "/home/kfuangsung/python_projects/algotrade/backtester.py", line 145, in b

  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
KeyboardInterrupt
KeyboardInterrupt
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/kfuangsung/anaconda3/envs/pyenv/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [ ]:
with pd.ExcelWriter('best_args.xlsx') as writer:  
    for k, v in ticker_args.items():
        v.to_excel(writer, sheet_name=k.replace('/','_'), index=False)